In [4]:
# Rename magasine column
import json
import pandas as pd

filepath = 'articles/NEI_articles.json'
with open(filepath, 'r', encoding='utf-8') as file:
    data = json.load(file)

for article in data:
    article['magasine'] = 'Nuclear Engineering International'

with open(filepath, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# Data exploration, check consistency etc.
df = pd.read_json('NCE_articles.json')
a = df['author'].unique()
for e in a:
    print(e)

In [120]:
import json
from datetime import datetime


# Check datetime format WNN
with open('articles/WNN_articles.json', 'r') as file:
    data = json.load(file)

dates = set()

for article in data:
    date_str = article.get('date', None)  # Assuming 'date' is the key for date field
    if date_str:
        try:
            # Convert string to datetime object if necessary
            date_obj = datetime.strptime(date_str, '%d %B %Y')  # Adjust the format if necessary
            dates.add(date_obj.date())  # Store only the date part
        except ValueError:
            # Handle the case where the date format is not as expected
            print(f"Unexpected date format: {date_str}")

# Print all unique dates
for date in sorted(dates):
    print(date)

Unexpected date format: Staff car park unveiled as Canadian MMR's planned location
Unexpected date format: China's demonstration HTR-PM enters commercial operation
2010-02-02
2010-07-14
2010-07-28
2010-10-28
2011-01-11
2011-02-02
2011-02-18
2011-04-13
2011-05-24
2012-01-12
2012-03-05
2012-04-12
2012-04-20
2012-05-18
2012-05-22
2012-07-19
2012-11-13
2012-11-21
2012-11-29
2012-12-06
2013-01-25
2013-03-13
2013-03-26
2013-05-17
2013-06-27
2013-06-28
2013-07-02
2013-07-30
2013-08-02
2013-08-06
2013-08-13
2013-11-14
2013-12-13
2014-01-15
2014-02-21
2014-02-27
2014-03-05
2014-04-14
2014-05-29
2014-07-15
2014-09-09
2014-09-22
2014-12-15
2015-02-03
2015-02-11
2015-03-04
2015-03-18
2015-03-31
2015-04-13
2015-04-21
2015-06-15
2015-07-02
2015-07-30
2015-08-03
2015-08-07
2015-09-04
2015-10-05
2015-10-21
2015-10-26
2015-11-09
2015-11-26
2015-12-03
2016-01-07
2016-01-08
2016-01-13
2016-01-15
2016-01-21
2016-02-19
2016-03-03
2016-03-18
2016-04-18
2016-04-28
2016-05-04
2016-05-16
2016-07-11
2016-07-13


In [123]:
# Merging articles
import glob
import json

json_files = glob.glob('articles/*.json')

all_articles = []

for filepath in json_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
        all_articles.extend(data)

with open('articles/240627_all_articles.json', 'w', encoding='utf-8') as file:
    json.dump(all_articles, file, ensure_ascii=False, indent=4)

In [124]:
# Data exploration
import pandas as pd
import random
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

df = pd.read_json('articles/240627_all_articles.json')
#df['date'] = pd.to_datetime(df['date'])

In [112]:
def count_words(paragraphs):
    if paragraphs is None:
        return 0
    return sum(len(paragraph.split()) for paragraph in paragraphs if paragraph)

def get_stats(data):
    rows = f'{data.shape[0]:,}'
    rows2 = data.shape[0]
    cols = f'{data.shape[1]:,}'
    earliest = min(data['date']).date()
    latest = max(data['date']).date()
    num_words_total = sum(data['num_words'])

    article_breakdown = data.groupby('magasine').agg(
        num_articles=('text', 'size'),
        total_words=('num_words', 'sum')
    ).reset_index()
    article_breakdown['percentage_articles'] = (article_breakdown['num_articles'] / rows2) * 100
    article_breakdown['percentage_words'] = (article_breakdown['total_words'] / num_words_total) * 100
    
    num_words_total = sum(data['num_words'])

    # Format numbers
    article_breakdown['num_articles'] = article_breakdown['num_articles'].apply(lambda x: f"{x:,}")
    article_breakdown['total_words'] = article_breakdown['total_words'].apply(lambda x: f"{x:,}")
    article_breakdown['percentage_articles'] = article_breakdown['percentage_articles'].apply(lambda x: f"{x:.2f}%")
    article_breakdown['percentage_words'] = article_breakdown['percentage_words'].apply(lambda x: f"{x:.2f}%")
    num_words_formatted = f'{num_words_total:,}'

    print(f'Total num articles: {rows:>20}')
    print(f'Total num words: {num_words_formatted:>23}')
    print(f'Earliest date: {str(earliest):>25}')
    print(f'Latest date: {str(latest):>27}\n')
    print(article_breakdown)

def split_data(data, training_percent):
    length = data.shape[0]
    random_order = random.sample(range(length), length)
    train_order = random_order[:training_percent]

In [125]:
df['num_words'] = df['text'].apply(count_words)

cutoff = '2021-06-14' # Exactly 2 years
df_new = df[df['date'] >= cutoff]
df_new.loc[:,'num_words'] = df_new['text'].apply(count_words)
get_stats(df_new)

AttributeError: 'str' object has no attribute 'date'